In [50]:
import matplotlib.pyplot as plt

import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader

from tqdm import trange
import numpy as np
import os
import random

# (0) Butterfly Data

In [44]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset Class
class ImageDataset(Dataset):
    def __init__(self, path_label, transform=None):
        self.path_label = path_label
        self.transform = transform

    def __len__(self):
        return len(self.path_label)

    def __getitem__(self, idx):
        path, label = self.path_label[idx]
        img = Image.open(path).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)

        return img, label
    
# Create dataset from raw structure obtained from Kaggle
def create_dataset(path = './data/butterfly/'):
    train_path = path + 'train'
    test_path = path + 'test'
    
    class_names=sorted(os.listdir(train_path))
    N=list(range(len(class_names)))
    normal_mapping=dict(zip(class_names,N)) 
    reverse_mapping=dict(zip(N,class_names))

    paths0=[]
    for dirname, _, filenames in os.walk(train_path):
        for filename in filenames:
            if filename[-4:]=='.jpg':
                path=os.path.join(dirname, filename)
                label=dirname.split('/')[-1]
                if label == '.ipynb_checkpoints':
                    continue
                paths0+=[(path,normal_mapping[label])]
            
    tpaths0=[]
    for dirname, _, filenames in os.walk(test_path):
        for filename in filenames:
            if filename[-4:]=='.jpg':
                path=os.path.join(dirname, filename)
                label=dirname.split('/')[-1]
                if label == '.ipynb_checkpoints':
                    continue
                tpaths0+=[(path,normal_mapping[label])]

    random.shuffle(paths0)            
    random.shuffle(tpaths0)  

    
    trainset = ImageDataset(paths0, transform)
    testset = ImageDataset(tpaths0, transform)
    
    return trainset, testset, normal_mapping, reverse_mapping


trainset, testset, normal_mapping, reverse_mapping = create_dataset()
assert len(trainset) == 12594, 'Size of train set not match'
assert len(testset) == 500, 'Size of test set not match'

# (1) Classifier

In [47]:
# Convolutional block with layer norm
class ConvBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dropout, padding=0, stride=1, bias=True):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, bias=bias)
        self.norm = nn.BatchNorm2d(num_features=out_channels)
        self.dropout = nn.Dropout2d(p=dropout)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.norm(x)
        x = self.dropout(x)
        return x

# Simplified VGG classifier
class Classifier(torch.nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv1 = ConvBlock(in_channels=in_channels, out_channels=64, kernel_size=3, dropout=0, padding=1, stride=1, bias=True)
        self.conv2 = ConvBlock(in_channels=64, out_channels=128, kernel_size=3, dropout=0, padding=1, stride=1, bias=True)
        self.conv3 = ConvBlock(in_channels=128, out_channels=256, kernel_size=3, dropout=0, padding=1, stride=1, bias=True)
        self.conv4 = ConvBlock(in_channels=256, out_channels=512, kernel_size=3, dropout=0, padding=1, stride=1, bias=True)
        self.conv5 = ConvBlock(in_channels=512, out_channels=512, kernel_size=3, dropout=0, padding=1, stride=1, bias=True)
        self.fc = torch.nn.LazyLinear(out_features=num_classes)
        
    def forward(self, x):
        x = F.silu(self.conv1(x))
        x = F.max_pool2d(x ,kernel_size=2)
        
        x = F.silu(self.conv2(x))
        x = F.max_pool2d(x ,kernel_size=2)
        
        x = F.silu(self.conv3(x))
        x = F.max_pool2d(x ,kernel_size=2)
        
        x = F.silu(self.conv4(x))
        x = F.max_pool2d(x ,kernel_size=2)
        
        x = F.silu(self.conv4(x))
        x = F.max_pool2d(x ,kernel_size=2)
        
        x = x.view(x.size()[0], -1)
        logits = self.fc(x)
        return logits
    

In [55]:
def train(model, train_loader, optimizer, criterion, epoch, device):
    model = model.to(device)
    criterion = criterion.to(device)
    
    train_loss = 0
    model.train()
    for batch_idx, (inp, label) in enumerate(train_loader):
        optimizer.zero_grad()
        inp, label = inp.to(device), label.to(device)
        output = model(inp)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        if batch_idx % (len(train_loader)//2) == 0:
            print('Train({})[{:.0f}%]: Loss: {:.4f}'.format(
                epoch, 100. * batch_idx / len(train_loader), train_loss/(batch_idx+1)))
            
batch_size=50
lr = 0.001
device='cuda'
train_loader = DataLoader(dataset=trainset, batch_size=batch_size)
model = Classifier(in_channels=3, num_classes=len(normal_mapping))
optimizer = torch.optim.AdamW(model.parameters(),lr=lr)
criterion = nn.CrossEntropyLoss()

/Users/kwongyuchong/anaconda3/envs/mlp/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [56]:
for epoch in trange(1, 10 + 1):
    train(model, train_loader, optimizer, criterion, epoch, device=device)
    scheduler.step()
torch.save(model.state_dict(), 'butterfly_classifier.pth')

  0%|          | 0/10 [00:00<?, ?it/s]


AssertionError: Torch not compiled with CUDA enabled